In [1]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on 
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  1  955k    1 14323    0     0  24687      0  0:00:39 --:--:--  0:00:39 24780
 48  955k   48  463k    0     0   324k      0  0:00:02  0:00:01  0:00:01  325k
100  955k  100  955k    0     0   597k      0  0:00:01  0:00:01 --:--:--  598k


In [1]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [2]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [3]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [4]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [7]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)
    
    def predict(self, user, item):
        return self.forward(user, item)

In [8]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()
        
        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        #--- Producing new continuous IDs for users and movies ---
        
        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}
        
        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}
        
        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        
        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [9]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: True
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0069, 0.0212, 0.0058,  ..., 0.0067, 0.0007, 0.0215],
        [0.0312, 0.0343, 0.0195,  ..., 0.0337, 0.0422, 0.0430],
        [0.0244, 0.0383, 0.0429,  ..., 0.0450, 0.0019, 0.0284],
        ...,
        [0.0013, 0.0125, 0.0048,  ..., 0.0497, 0.0447, 0.0061],
        [0.0401, 0.0168, 0.0425,  ..., 0.0240, 0.0194, 0.0360],
        [0.0019, 0.0247, 0.0121,  ..., 0.0333, 0.0037, 0.0158]])
item_factors.weight tensor([[0.0299, 0.0500, 0.0279,  ..., 0.0052, 0.0327, 0.0182],
        [0.0369, 0.0002, 0.0129,  ..., 0.0318, 0.0317, 0.0064],
        [0.0278, 0.0442, 0.0163,  ..., 0.0345, 0.0303, 0.0092],
        ...,
        [0.0141, 0.0373, 0.0249,  ..., 0.0367, 0.0349, 0.0348],
        [0.0075, 0.0114, 0.0492,  ..., 0.0422, 0.0107, 0.0134],
        [0.0106, 0.0189, 0.0288,  ..., 0.0008, 0.0129, 0.0179]])


In [10]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

C:\Users\abdir\AppData\Local\Temp\ipykernel_17308\59576823.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: 11.063584843262802
iter #1 Loss: 4.744082423333589
iter #2 Loss: 2.4729473518236036
iter #3 Loss: 1.7201574590000404
iter #4 Loss: 1.345625264012269
iter #5 Loss: 1.128412736248849
iter #6 Loss: 0.9915965710799706
iter #7 Loss: 0.9002187918436709
iter #8 Loss: 0.8372296274162186
iter #9 Loss: 0.7924836054066111
iter #10 Loss: 0.7593924434959586
iter #11 Loss: 0.7349380469564254
iter #12 Loss: 0.7159714663240511
iter #13 Loss: 0.7015181594920642
iter #14 Loss: 0.6907402070494473
iter #15 Loss: 0.6819420863075305
iter #16 Loss: 0.6747928611215601
iter #17 Loss: 0.6700583217561548
iter #18 Loss: 0.6657503828179413
iter #19 Loss: 0.6627726240962895
iter #20 Loss: 0.6607759078293283
iter #21 Loss: 0.6590185140065735
iter #22 Loss: 0.6577540036990558
iter #23 Loss: 0.6568479255446927
iter #24 Loss: 0.6559087094392269
iter #25 Loss: 0.6550561811415677
iter #26 Loss: 0.6541972075622093
iter #27 Loss: 0.6534011369656185
iter #28 Loss: 0.6525897617796956
iter #29 Loss: 0.6514357901

In [11]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0 
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[ 1.1512,  1.3940,  1.1673,  ...,  1.2324,  0.7211,  1.7474],
        [ 1.2859,  1.1964,  2.3304,  ..., -0.0715,  0.2629,  1.8346],
        [-1.7054, -1.6825,  2.8490,  ...,  2.0377,  0.3576,  0.5939],
        ...,
        [ 1.4475,  0.2992, -0.2271,  ...,  2.4767,  2.1634, -0.0642],
        [ 0.7949,  0.8243,  1.0827,  ...,  1.0353,  1.4539,  1.0300],
        [ 1.2303,  0.2946,  2.2650,  ...,  1.0810,  0.7876,  0.7840]],
       device='cuda:0')
item_factors.weight tensor([[0.8154, 0.6830, 0.6034,  ..., 0.1318, 0.3278, 0.3730],
        [0.1056, 0.8786, 0.6946,  ..., 0.3408, 0.2873, 0.7475],
        [0.6197, 0.7515, 0.7200,  ..., 0.4309, 0.4923, 0.0477],
        ...,
        [0.3231, 0.3208, 0.3338,  ..., 0.3444, 0.3440, 0.3437],
        [0.3888, 0.3733, 0.4296,  ..., 0.4212, 0.3944, 0.3949],
        [0.4120, 0.4073, 0.4326,  ..., 0.4057, 0.4133, 0.4213]],
       device='cuda:0')


In [12]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [13]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [14]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [15]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
    print("Cluster #{}".format(cluster))
    movs = []
    # Find movie indices belonging to the current cluster
    for movidx in np.where(kmeans.labels_ == cluster)[0]:
        movid = train_set.idx2movieid[movidx]
        # Check how many ratings this movie has
        rat_count = len(ratings_df.loc[ratings_df['movieId'] == movid])
        movs.append((movie_names[movid], rat_count))
    # Sort movies by rating count in descending order and print top 10
    for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
        print("\t", mov[0])

Cluster #0
	 True Lies (1994)
	 Speed (1994)
	 Mask, The (1994)
	 Mrs. Doubtfire (1993)
	 Die Hard: With a Vengeance (1995)
	 Home Alone (1990)
	 Jumanji (1995)
	 Clueless (1995)
	 Outbreak (1995)
	 Ace Ventura: When Nature Calls (1995)
Cluster #1
	 Batman Returns (1992)
	 Arachnophobia (1990)
	 Star Trek: The Motion Picture (1979)
	 Cell, The (2000)
	 Batman & Robin (1997)
	 Godzilla (1998)
	 Daredevil (2003)
	 Spy Hard (1996)
	 Spawn (1997)
	 Honey, I Blew Up the Kid (1992)
Cluster #2
	 Forrest Gump (1994)
	 Shawshank Redemption, The (1994)
	 Silence of the Lambs, The (1991)
	 Matrix, The (1999)
	 Star Wars: Episode IV - A New Hope (1977)
	 Braveheart (1995)
	 Terminator 2: Judgment Day (1991)
	 Toy Story (1995)
	 Star Wars: Episode V - The Empire Strikes Back (1980)
	 Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
Cluster #3
	 Schindler's List (1993)
	 Fargo (1996)
	 Monty Python and the Holy Grail (1975)
	 One Flew Over the Cuckoo's Nest (1975)
	 God